In [125]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [123]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
PATH_TO_DATA = r'E:\Work\Kaggle_competitions\catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2'

In [8]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [9]:
train_df.head(5)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [10]:
train_df.shape

(253561, 21)

In [11]:
test_df.head(5)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [12]:
test_df.shape

(82797, 20)

In [13]:
train_test_df = pd.concat([train_df, test_df])

In [151]:
train_test_df.shape

(336358, 21)

In [15]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')

In [16]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


In [153]:
y = train_df["target"]

In [154]:
y.value_counts()

0    251264
1      2297
Name: target, dtype: int64

In [19]:
def to_sparse(dense):
    dense = dense.values
    indptr = list(range(0, len(dense)* 10 + 1,  10))
    indices = list(np.concatenate(dense, axis = 0 ))
    data = list(np.ones( len(dense) * 10))
    X_sparse = csr_matrix((data, indices, indptr), dtype=int)
    return X_sparse

In [20]:
train_test_sparse = to_sparse(train_test_df_sites)


In [21]:
train_test_sparse 

<336358x48372 sparse matrix of type '<class 'numpy.int32'>'
	with 3363580 stored elements in Compressed Sparse Row format>

In [22]:
train_test_sparse = train_test_sparse[:,1:]

In [23]:
train_test_sparse 

<336358x48371 sparse matrix of type '<class 'numpy.intc'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [24]:
train_sparse = train_test_sparse[:253561,:]

In [25]:
train_sparse.shape

(253561, 48371)

In [26]:
test_sparse =  train_test_sparse[253561:, :]

In [27]:
test_sparse.shape

(82797, 48371)

In [28]:
X_train, X_valid, y_train, y_valid = train_test_split(train_sparse, y, test_size=0.3 , random_state= 17)

In [29]:
sgd_logit = SGDClassifier(loss = "log", random_state = 17, n_jobs = -1)
sgd_logit.fit(X_train, y_train)

SGDClassifier(loss='log', n_jobs=-1, random_state=17)

In [30]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

In [31]:
roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])

0.9428421084277627

In [32]:
sgd_logit = SGDClassifier(loss = "log", random_state = 17, n_jobs = -1).fit(train_sparse, y)
y_test_pred = sgd_logit.predict_proba(test_sparse)


In [33]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [34]:
write_to_submission_file(y_test_pred[:,1], r"E:\Work\Kaggle_competitions\catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2\baseline_submission",
                             target='target', index_label="session_id")

In [98]:
%%time
sgd_params = {'alpha': np.linspace(0.00001, 1000, 20)}

sgd_grid_searcher = GridSearchCV(estimator = sgd_logit, param_grid = sgd_params, return_train_score = True, scoring = "roc_auc")
sgd_grid_searcher.fit(X_train, y_train)

Wall time: 56 s


GridSearchCV(estimator=SGDClassifier(loss='log', n_jobs=-1, random_state=17),
             param_grid={'alpha': array([1.00000000e-05, 5.26315884e+01, 1.05263167e+02, 1.57894745e+02,
       2.10526324e+02, 2.63157902e+02, 3.15789481e+02, 3.68421059e+02,
       4.21052637e+02, 4.73684216e+02, 5.26315794e+02, 5.78947373e+02,
       6.31578951e+02, 6.84210529e+02, 7.36842108e+02, 7.89473686e+02,
       8.42105265e+02, 8.94736843e+02, 9.47368422e+02, 1.00000000e+03])},
             return_train_score=True, scoring='roc_auc')

In [99]:
sgd_grid_searcher.best_score_

0.9530202130571362

In [100]:
sgd_grid_searcher.best_params_

{'alpha': 1e-05}

'alpha': 1e-05

In [101]:
sgd_params1 = {'alpha': np.linspace(0.0000001, 0.0001, 40)}

sgd_grid_searcher1 = GridSearchCV(estimator = sgd_logit, param_grid = sgd_params1, return_train_score = True, scoring = "roc_auc")
sgd_grid_searcher1.fit(X_train, y_train)

GridSearchCV(estimator=SGDClassifier(loss='log', n_jobs=-1, random_state=17),
             param_grid={'alpha': array([1.00000000e-07, 2.66153846e-06, 5.22307692e-06, 7.78461538e-06,
       1.03461538e-05, 1.29076923e-05, 1.54692308e-05, 1.80307692e-05,
       2.05923077e-05, 2.31538462e-05, 2.57153846e-05, 2.82769231e-05,
       3.08384615e-05, 3.34000000e-05, 3.59615385e-05, 3.85230769e-05,
       4.108461...e-05,
       5.13307692e-05, 5.38923077e-05, 5.64538462e-05, 5.90153846e-05,
       6.15769231e-05, 6.41384615e-05, 6.67000000e-05, 6.92615385e-05,
       7.18230769e-05, 7.43846154e-05, 7.69461538e-05, 7.95076923e-05,
       8.20692308e-05, 8.46307692e-05, 8.71923077e-05, 8.97538462e-05,
       9.23153846e-05, 9.48769231e-05, 9.74384615e-05, 1.00000000e-04])},
             return_train_score=True, scoring='roc_auc')

In [102]:
sgd_grid_searcher1.best_score_

0.9524456943216165

In [103]:
sgd_grid_searcher1.best_params_

{'alpha': 1.803076923076923e-05}

In [104]:
estimator = sgd_grid_searcher1.best_estimator_

**Попытаемся использовать признаки, связанные со временем посещения сайтов**

In [37]:
train_test_df_times = train_test_df[['time%d' % i for i in range(1, 11)]].fillna(0)

In [38]:
train_test_df_times

,time1,time2,time3,time4,time5,time6,time7,time8,time9,time10
session_id,,,,,,,,,,
1,2014-02-20 10:02:45,0,0,0,0,0,0,0,0,0
2,2014-02-22 11:19:50,2014-02-22 11:19:50,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:51,2014-02-22 11:19:52,2014-02-22 11:19:52,2014-02-22 11:20:15,2014-02-22 11:20:16
3,2013-12-16 16:40:17,2013-12-16 16:40:18,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:19,2013-12-16 16:40:20,2013-12-16 16:40:21,2013-12-16 16:40:22,2013-12-16 16:40:24
4,2014-03-28 10:52:12,2014-03-28 10:52:42,2014-03-28 10:53:12,2014-03-28 10:53:42,2014-03-28 10:54:12,2014-03-28 10:54:42,2014-03-28 10:55:12,2014-03-28 10:55:42,2014-03-28 10:56:12,2014-03-28 10:56:42
5,2014-02-28 10:53:05,2014-02-28 10:55:22,2014-02-28 10:55:22,2014-02-28 10:55:23,2014-02-28 10:55:23,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:55:59,2014-02-28 10:57:06,2014-02-28 10:57:11
...,...,...,...,...,...,...,...,...,...,...
82793,2014-10-02 18:20:09,2014-10-02 18:20:09,2014-10-02 18:20:09,0,0,0,0,0,0,0
82794,2014-05-26 14:16:40,2014-05-26 14:16:41,2014-05-26 14:16:44,2014-05-26 14:16:44,2014-05-26 14:17:19,2014-05-26 14:17:19,2014-05-26 14:17:19,2014-05-26 14:17:19,2014-05-26 14:17:19,2014-05-26 14:17:19
82795,2014-05-02 11:21:56,2014-05-02 11:21:56,2014-05-02 11:21:56,2014-05-02 11:22:03,2014-05-02 11:22:03,2014-05-02 11:22:03,2014-05-02 11:22:03,2014-05-02 11:22:04,2014-05-02 11:22:04,2014-05-02 11:22:04


Преобразуем элементы датафрейма в формат datatime

In [87]:
%%time
train_test_df_times = train_test_df_times.mask(train_test_df_times != 0, train_test_df_times.apply(lambda x: pd.to_datetime(x)))

Wall time: 25.4 s


In [ ]:
type(train_test_df_times.iloc[0, 1])

In [ ]:
train_test_df_times.iloc[0, :]

In [ ]:
(train_test_df_times.iloc[0, :] == 0).astype(int).sum(axis=1)


In [114]:
def feature_creation(df):
    new_features = pd.DataFrame()
    avg_time_on_site = 60
    for i in range(0, len(df)):
        row = df.iloc[i, :].to_numpy()
        if np.count_nonzero(row) < 10:
            session_length = np.count_nonzero(row)
        if np.count_nonzero(row) == 10:
            session_length = 10
        avg_time_on_site_array = []
        for i in range(1, session_length - 1):
            avg_time_on_site_array.append((row[i] - row[i-1]).total_seconds()) 
        if len(avg_time_on_site_array) > 1:
            avg_time_on_site = int(sum(avg_time_on_site_array) / len(avg_time_on_site_array))
        if len(avg_time_on_site_array) == 1:
            avg_time_on_site = avg_time_on_site_copy
        avg_time_on_site_copy = avg_time_on_site
        if session_length == 10:
            session_timestamp = (row[9] - row[0]).total_seconds()
        if session_length != 0:
            session_timestamp = (row[session_length - 1] - row[0]).total_seconds()
        start_hour = row[0].hour
        day_of_week = row[0].weekday()
        new_features = new_features.append({"avg_time_on_site": avg_time_on_site, "session_timestamp": session_timestamp,
                          "start_hour": start_hour ,"day_of_week": day_of_week}, ignore_index=True)
    return new_features

In [115]:
 new_features = feature_creation(train_test_df_times)

In [118]:
new_features

,avg_time_on_site,session_timestamp,start_hour,day_of_week
0,60.0,0.0,10.0,3.0
1,3.0,26.0,11.0,5.0
2,0.0,7.0,16.0,0.0
3,30.0,270.0,10.0,4.0
4,30.0,246.0,10.0,4.0
...,...,...,...,...
336353,32.0,0.0,18.0,3.0
336354,4.0,39.0,14.0,0.0
336355,1.0,8.0,11.0,4.0
336356,1.0,13.0,10.0,5.0


In [119]:
train_test_sparse

<336358x48371 sparse matrix of type '<class 'numpy.intc'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [120]:
 new_features

,avg_time_on_site,session_timestamp,start_hour,day_of_week
0,60.0,0.0,10.0,3.0
1,3.0,26.0,11.0,5.0
2,0.0,7.0,16.0,0.0
3,30.0,270.0,10.0,4.0
4,30.0,246.0,10.0,4.0
...,...,...,...,...
336353,32.0,0.0,18.0,3.0
336354,4.0,39.0,14.0,0.0
336355,1.0,8.0,11.0,4.0
336356,1.0,13.0,10.0,5.0


In [121]:
train_test_sparse

<336358x48371 sparse matrix of type '<class 'numpy.intc'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [129]:
ohe = OneHotEncoder()

In [130]:
OHE_features = ohe.fit_transform(new_features.iloc[:, 2:])

In [144]:
OHE_features.shape

(336358, 24)

In [139]:
from sklearn.preprocessing import StandardScaler
from scipy.sparse import coo_matrix, hstack
scaler = StandardScaler()

In [140]:
scaled_features = scaler.fit_transform(new_features.iloc[:, :2])

In [142]:
X_new_fe =  hstack((train_test_sparse, OHE_features, scaled_features))

In [157]:
train_test_sparse

<336358x48371 sparse matrix of type '<class 'numpy.intc'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [158]:
X_new_fe = csr_matrix(X_new_fe)

In [159]:
X_new_fe 

<336358x48397 sparse matrix of type '<class 'numpy.float64'>'
	with 3212330 stored elements in Compressed Sparse Row format>

In [155]:
y

session_id
1         0
2         0
3         0
4         0
5         0
         ..
253557    0
253558    0
253559    0
253560    0
253561    0
Name: target, Length: 253561, dtype: int64

In [161]:
sgd_params3 = {'alpha': np.linspace(0.00001, 1000, 20)}

sgd_grid_searcher3 = GridSearchCV(estimator = sgd_logit, param_grid = sgd_params3, return_train_score = True, scoring = "roc_auc")
sgd_grid_searcher3.fit(X_new_fe[:253561, :], y)

GridSearchCV(estimator=SGDClassifier(loss='log', n_jobs=-1, random_state=17),
             param_grid={'alpha': array([1.00000000e-05, 5.26315884e+01, 1.05263167e+02, 1.57894745e+02,
       2.10526324e+02, 2.63157902e+02, 3.15789481e+02, 3.68421059e+02,
       4.21052637e+02, 4.73684216e+02, 5.26315794e+02, 5.78947373e+02,
       6.31578951e+02, 6.84210529e+02, 7.36842108e+02, 7.89473686e+02,
       8.42105265e+02, 8.94736843e+02, 9.47368422e+02, 1.00000000e+03])},
             return_train_score=True, scoring='roc_auc')

In [163]:
sgd_grid_searcher3.best_score_

0.9852751445296015

In [165]:
sgd_grid_searcher3.best_params_

{'alpha': 1e-05}

In [170]:
n_components = np.linspace(100, 30000, 10)

In [176]:
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate
from sklearn.decomposition import TruncatedSVD

In [182]:
train_test_sparse = train_test_sparse.astype(int)

In [183]:
train_test_sparse

<336358x48371 sparse matrix of type '<class 'numpy.intc'>'
	with 3195430 stored elements in Compressed Sparse Row format>

In [185]:
for num_components in n_components:
    svd = TruncatedSVD(n_components = num_components)
    X_old_new =  svd.fit(train_test_sparse)
    X_new = hstack(X_old_new, OHE_features, scaled_features)
    cv_results = cross_validate(sgd_logit, X_new_fe[:253561, :], y)
    print("N_components = " + str(num_components))
    print("cv_scores  = " + str(cv_results['test_score']))

TypeError: 'numpy.float64' object cannot be interpreted as an integer